In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#합성곱 함수 구현
def conv(a,b):
    c = np.array(a)*np.array(b)
    return np.sum(c)

In [ ]:
#Maxpooling 함수 구현(한 개의 map 계산)
def MaxPooling(nimg):
    nimg = np.array(nimg)
    i0, j0 = nimg.shape
    i1 = int((i0+1)/2)
    j1 = int((j0+1)/2)
    output = np.zeros((i1,j1))
    
    if i0%2 ==1:
        i0+=1
        tmp = np.zeros((1,j0))
        nimg = np.concatenate([nimg,tmp], axis=0)
        
    if j0%2 ==1:
        i0+=1
        tmp = np.zeros((i0,1))
        nimg = np.concatenate([nimg,tmp], axis=1)
        
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            a = np.array(nimg[2*i:2*i+2,2*j:2*j+2])
            output[i,j] = a.max()
    
    return output
        

In [ ]:
#합성곱 출력 층(feature map)함수 구현(한 개의 filter 계산)
def featuring(nimg, filters):
    feature = np.zeros((nimg.shape[0]-2, nimg.shape[1]-2))
    for i in range(feature.shape[0]):
        for j in range(feature.shape[1]):
            a = nimg[i:i+3, j:j+3,]
            feature[i,j] = conv(a, filters)
    
    return output

In [ ]:
#Maxpooling 출력 층 함수 구현(여러 map 계산)
def Pooling(nimg):
    nimg = np.array(nimg)
    pool0 = []
    for i in range(len(nimg)):
        pool0.append(MaxPooling(nimg[i]))
    return pool0

In [ ]:
def to_img(nimg):
    nimg = np.array(nimg)
    nimg = np.uint8(np.round(nimg))
    fimg = []
    for i in range(len(nimg)):
        fimg.append(Image.fromarray(nimg[i]))
    return fimg

In [ ]:
#feature map 생성(여러 filter 계산)
def ConvD(nimg):
    nimg = np.array(nimg)
    feat0 = []
    for i in range(len(filter)):
        feat0.append(featuring(nimg,filter[i]))
    return feat0

In [ ]:
#ReLU 활성화 함수

def ReLU(f0):
    f0 = np.array(f0)
    f0 = (f0>0)*f0
    return f0

In [ ]:
#CNN Layer 함수 : Conv + ReLU + Maxpooling
def ConvMax(nimg):
    nimg = np.array(nimg)
    f0 = ConvD(nimg)
    f0 = ReLu(f0)
    fg = Pooling(f0)
    return f0, fg

In [ ]:
#그림 그리기 : 합성곱 후의 상태와 MaxPooling 후의 상태를 그림으로 그리기
def draw(f0, fg0, size = (12,8), k=-1):
    plt.figure(figsize = size)
    
    for i in range(len(f0)):
        plt.subplot(2, len(f0),i+1)
        plt.gca().set_title('Conv'+str(k)+'-'+str(i))
        plt.imshow(f0[i])
        
    for i in range(len(fg0)):
        plt.subplot(2, len(fg0),len(f0)+i+1)
        plt.gca().set_title('MaxP'+str(k)+'-'+str(i))
        plt.imshow(fg0[i])
        
    if k!=-1:
        plt.savefig('conv'+str(k)+'.png')

In [ ]:
#3개의 activation map 합치기 : MaxPooling 후의 결과 map들을 하나의 데이터로 통합
def join(mm):
    mm = np.array(mm)
    ml = np.zeros((mm.shape[1],mm.shape[2],mm.shape[0]))
    for i in range(mm.shape[1]):
        for j in range(mm.shape[2]):
            for k in range(mm.shape[0]):
                ml[i][j][k] = mm[k][i][j]
                
    return ml

In [ ]:
#CNN Layer 과정을 계산하고 결과를 그림으로 출력
def ConvDraw(p0,size=(12,8), k=-1):
    f0, fg0 = ConvMax(p0)
    f0 = to_img(f0)
    fg1 = to_img(fg0)
    draw(f0, fg1, size, k)
    p1 = join(fg0)
    return p1

#m0 = ConvDraw(nimg31,(12,10), 0)